In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from sympy.utilities.lambdify import implemented_function
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# Subbing functions

In [ ]:
y,mu,sigma=sym.symbols('y \mu \sigma')
L=-1/2*(sym.log(2*sym.pi)+sym.log(sigma**2)+(y-mu)**2/sigma**2)
Lmu=sym.diff(L,mu)
Lsigma=sym.diff(L,sigma)
L_lambda=sym.lambdify([y,mu,sigma],L,modules=['numpy'])
Lmu_lambda=sym.lambdify([y,mu,sigma],Lmu,modules=['numpy'])
Lsigma_lambda=sym.lambdify([y,mu,sigma],Lsigma,modules=['numpy'])

# fixing sigma at a point


In [ ]:
sym.lambdify([mu,sigma],L.subs(y,0.00005),modules=['numpy'])(1,2)

# Subbing data range obtained from submission

In [ ]:
mu_numerics=np.linspace(0.000001,0.00299,1000)
sigma_numerics=np.linspace(0.000001,0.000155,1000)
y_numerics=0.001*np.ones(sigma_numerics.shape)
L_range=L_lambda(y_numerics,mu_numerics,sigma_numerics)
Lmu_range=Lmu_lambda(y_numerics,mu_numerics,sigma_numerics)
Lsigma_range=Lsigma_lambda(y_numerics,mu_numerics,sigma_numerics)

In [ ]:
XX,YY=np.meshgrid(mu_numerics,sigma_numerics)
ZZ=sym.lambdify([mu,sigma],L.subs(y,0.00002),modules=['numpy'])(XX,YY)

# Plot

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Surface(
    x = mu_numerics,
    y = sigma_numerics,
    z = ZZ))
fig.update_layout(
    title="Plot Title",
    
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

# Four unknown approach, assuming there is mu_ideal,sigma_ideal,mu_ref, sigma_ref

In [ ]:


# Define the unknowns (shared across all equations)
sigma_ideal, mu_ideal, sigma_ref, mu_ref= sym.symbols('sigma_ideal mu_ideal sigma_ref mu_ref ')

# Define the known scores (replace these with actual values)
scores = [0.545, 0.537, 0.514, 0.536]

# Define the GLL equation for ideal and reference, rounding each step to 9 decimal places
def GLL(score, mu_user, sigma_user, mu_ideal, sigma_ideal, mu_ref, sigma_ref ):
    # Define the equation symbolically, rounding at each key step
    equation = sym.Eq(
        sym.N(-2 * score * 283 ),
        (
            + sym.N(sym.log(sigma_user**2,10) )
            + sym.N(((0.00281366035038192 - mu_user)**2) / sigma_user**2)
            - sym.N(sym.log(sigma_ref**2,10) )
            - sym.N(((0.00281366035038192 - mu_ref)**2) / sigma_ref**2 )
        ) / (
            + sym.N(sym.log(sigma_ideal**2,10) )
            + sym.N(((0.00281366035038192 - mu_ideal)**2) / sigma_ideal**2)
            - sym.N(sym.log(sigma_ref**2,10))
            - sym.N(((0.00281366035038192 - mu_ref)**2) / sigma_ref**2)
        )
    )
    return equation

# Score equations (shared L_ideal and L_ref)
eq1 = GLL(scores[0], 0.0027237, 0.0001433, mu_ideal, sigma_ideal, mu_ref, sigma_ref )
eq2 = GLL(scores[1], 0.0027503, 0.0001496, mu_ideal, sigma_ideal, mu_ref, sigma_ref )
eq3 = GLL(scores[2], 0.0026550000000000054, 0.000176, mu_ideal, sigma_ideal, mu_ref, sigma_ref )
eq4 = GLL(scores[3], 0.0028578, 0.0001510, mu_ideal, sigma_ideal, mu_ref, sigma_ref )

# Initial guesses for sigma_ideal, mu_ideal, sigma_ref, mu_ref, y (must provide initial guesses for nsolve)


# Solve numerically using nsolve
print(sym.N(sym.log(0.00016**2,10)))
print(eq1)
print(eq2)
print(eq3)
print(eq4)
#solution = sym.solve([eq1, eq2, eq3, eq4], (sigma_ideal, mu_ideal, sigma_ref, mu_ref ))
 




# Three unknown approach, finding L_ref, L_ideal and y

In [ ]:
# Define the unknowns (shared across all equations)
L_ideal, L_ref, y = sym.symbols('L_ideal L_ref y')

# Define the known scores (replace these with actual values)
scores = [0.545, 0.537, 0.536]

# Define the GLL equation for ideal and reference, rounding each step to 9 decimal places
def GLL(score, mu_user, sigma_user, L_ideal, L_ref, y):
    # Define the equation symbolically, rounding at each key step
    equation = sym.Eq(
        sym.N(-2 * score * 283 ),
        (
            + sym.N(sym.log(sigma_user**2,10) )
            + sym.N(((y - mu_user)**2) / sigma_user**2)
            - L_ref
        ) / (
            + L_ideal
            - L_ref
        )
    )
    return equation

# Score equations (shared L_ideal and L_ref)
eq1 = GLL(scores[0], 0.0027237, 0.0001433, L_ideal, L_ref, y)
eq2 = GLL(scores[1], 0.0027495, 0.0001304, L_ideal, L_ref, y)
eq3 = GLL(scores[2], 0.0028578, 0.0001510, L_ideal, L_ref, y)

print(eq1)
print(eq2)
print(eq3)

solution = sym.solve([eq1, eq2, eq3], (L_ideal, L_ref, y))
print(solution)

# Finding mu_ideal, sigma_ideal

In [ ]:
# Define the unknowns (shared across all equations)
sigma_ideal, mu_ideal= sym.symbols('sigma_ideal mu_ideal ')

# Define the known scores (replace these with actual values)
scores = [0.545, 0.537]

# Define the GLL equation for ideal and reference, rounding each step to 9 decimal places
def GLL(score, mu_user, sigma_user, mu_ideal, sigma_ideal):
    # Define the equation symbolically, rounding at each key step
    equation = sym.Eq(
        sym.N(-2 * score * 283 ),
        (
            + sym.N(sym.log(sigma_user**2,10) )
            + sym.N(((0.00281366035038192 - mu_user)**2) / sigma_user**2)
            - (-23.2312173430632)
        ) / (
            + sym.N(sym.log(sigma_ideal**2,10) )
            + sym.N(((0.00281366035038192 - mu_ideal)**2) / sigma_ideal**2)
            -(-23.2312173430632)
        )
    )
    return equation

# Score equations (shared L_ideal and L_ref)
eq1 = GLL(scores[0], 0.0027237, 0.0001433, mu_ideal, sigma_ideal) #, mu_ref, sigma_ref )
eq2 = GLL(scores[1], 0.0027503, 0.0001496, mu_ideal, sigma_ideal) #, mu_ref, sigma_ref )
#eq3 = GLL(scores[2], 0.0026550000000000054, 0.000176, mu_ideal, sigma_ideal, mu_ref, sigma_ref )
#eq4 = GLL(scores[3], 0.0028578, 0.0001510, mu_ideal, sigma_ideal, mu_ref, sigma_ref )

# Initial guesses for sigma_ideal, mu_ideal, sigma_ref, mu_ref, y (must provide initial guesses for nsolve)


# Solve numerically using nsolve
print(sym.N(sym.log(0.00016**2,10)))
print(eq1)
print(eq2)
solution = sym.solve([eq1, eq2], (sigma_ideal, mu_ideal ))
print(solution)
 




# Finding mu_ref, sigma_ref

In [ ]:
# Define the unknowns (shared across all equations)
sigma_ref, mu_ref= sym.symbols('sigma_ref mu_ref ')

# Define the known scores (replace these with actual values)
scores = [0.545, 0.537]

# Define the GLL equation for ideal and reference, rounding each step to 9 decimal places
def GLL(score, mu_user, sigma_user,mu_ref, sigma_ref ):
    equation = sym.Eq(
        sym.N(-2 * score * 283 ),
        (
            + sym.N(sym.log(sigma_user**2,10) )
            + sym.N(((0.00281366035038192 - mu_user)**2) / sigma_user**2)
            - sym.N(sym.log(sigma_ref**2,10) )
            - sym.N(((0.00281366035038192 - mu_ref)**2) / sigma_ref**2 )
        ) / (
            + (-23.2828846457651)
            - sym.N(sym.log(sigma_ref**2,10))
            - sym.N(((0.00281366035038192 - mu_ref)**2) / sigma_ref**2)
        )
    )
    return equation

# Score equations (shared L_ideal and L_ref)
eq1 = GLL(scores[0], 0.0027237, 0.0001433, mu_ref, sigma_ref )
eq2 = GLL(scores[1], 0.0027503, 0.0001496, mu_ref, sigma_ref )

# Solve numerically using nsolve
print(sym.N(sym.log(0.00016**2,10)))
print(eq1)
print(eq2)

solution = sym.solve([eq1, eq2], (sigma_ref, mu_ref ))

print(solution)




# Finding mu_ideal, mu_sigma using L_ideal

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Define the function that represents the equation
def GLL_numeric(sigma_ideal):
    return (
        np.log10(2 * np.pi) + 
        np.log10(sigma_ideal**2) + 
        ((0.00281366035038192 - sigma_ideal * 19)**2) / sigma_ideal**2
    ) - 46.6  # The equation rearranged to be f(sigma_ideal) = 0

# Use fsolve to find the root, with an initial guess (e.g., 1)
initial_guess = 0.0001  # You can try other guesses like 0.1, 100, etc.
solution = fsolve(GLL_numeric, initial_guess)

print("Solution for sigma_ideal:", solution[0])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the function without equating to 46.6, so we can plot it
def GLL_numeric(sigma_ideal):
    return (
        np.log10(2 * np.pi) +
        np.log10(sigma_ideal**2) +
        ((0.00281366035038192 - sigma_ideal * 19)**2) / sigma_ideal**2
    )

# Create a range of values for sigma_ideal to plot
sigma_values = np.linspace(0.00000001, 50, 1000)
y_values = GLL_numeric(sigma_values)

# Plot the function
plt.plot(sigma_values, y_values, label="GLL(sigma_ideal)")
plt.axhline(46.6, color='red', linestyle='--', label="46.6")
plt.xlabel('sigma_ideal')
plt.ylabel('GLL(sigma_ideal)')
plt.title('Graph of GLL(sigma_ideal)')
plt.legend()
plt.grid(True)
plt.show()
